In [3]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd 
import time

In [ ]:
links = []

for page in range(1, 57):
    driver = webdriver.Chrome()
    driver.get(f'https://winmart.onl/shop/winmart-officialp01.html?trang={page}&page_num=100')

    # Chờ tối đa 10 giây để các div item-category-des xuất hiện
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "item-category-des"))
    )

    soup = BeautifulSoup(driver.page_source, 'html.parser')


    for div in soup.find_all('div', class_='item-category-des'):
        a_tag = div.find('a')
        if a_tag and a_tag.get('href'):
            links.append(a_tag['href'])

    driver.quit()
print(links)
print("Total:", len(links))


['https://winmart.onl/combo-6-muong-cafe-inox-116cm-lucky-star-vi-2-cai-mnls-364.html', 'https://winmart.onl/combo-12-mut-rua-ly-tach-poly-tech-kz118-482.html', 'https://winmart.onl/10-quyen-so-a3-bia-cung-6-tap-bang-dau-240-trang-5781.html', 'https://winmart.onl/4-can-nuoc-giat-d-nee-thai-lan-3l-5358.html', 'https://winmart.onl/2-chai-xi-dau-cang-cua-dac-biet-3079.html', 'https://winmart.onl/150-cai-bao-dung-the-nhan-vien-108-2118.html', 'https://winmart.onl/60-tap-truyen-tranh-tam-quoc-dien-nghia-3870.html', 'https://winmart.onl/dung-phe-binh-toi-3766.html', 'https://winmart.onl/2-chai-ruou-vang-hibiscus-roselle-914.html', 'https://winmart.onl/ruou-ban-ngot-hibiscus-roselle-915.html', 'https://winmart.onl/tam-suoi-dien-hk25-5493.html', 'https://winmart.onl/quat-suoi-fairlady-nsb-90-cong-nghe-duc-3923.html', 'https://winmart.onl/combo-8-chai-nuoc-giat-chong-phai-mau-kolortex-tang-2-chai-xa-vai-1-chai-lau-san-1-chai-lau-bep-229.html', 'https://winmart.onl/combo-4-chai-nuoc-giat-chong-l

In [ ]:
# Tạo DataFrame 
df = pd.DataFrame({
    "link": links
})

# Lưu file CSV
df.to_csv("products/product_links.csv", index=False, encoding="utf-8")

print("Saved product_links.csv thành công!")

Saved product_links.csv thành công!


In [2]:
df = pd.read_csv("products/product_links.csv")

data = []
driver = webdriver.Chrome()

for link in df['link']:
    print("Crawling:", link)
    driver.get(link)

    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, ".col-sm-5.mnv-product-details, .col-sm-6.mnv-product-details")
            )
        )
        time.sleep(1)
    except:
        print("⚠ Timeout:", link)
        continue

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Cả 2 trường hợp 5 & 6
    product_section = soup.select_one(".col-sm-5.mnv-product-details, .col-sm-6.mnv-product-details")
    if not product_section:
        print("❌ Không tìm thấy phần chi tiết sản phẩm → bỏ qua")
        continue

    name = product_section.find("h1").get_text(strip=True)

    price_new = soup.select_one(".price-new")
    price_new = price_new.get_text(strip=True) if price_new else ""

    product_code = soup.select_one("#p_model")
    product_code = product_code.get_text(strip=True) if product_code else ""

    data.append({
        "name": name,
        "price": price_new,
        "code": product_code
    })

driver.quit()

df_out = pd.DataFrame(data)
df_out.to_csv("products/product_details.csv", index=False, encoding="utf-8-sig")
print("✅ Done -> products/product_details.csv")

Crawling: https://winmart.onl/combo-6-muong-cafe-inox-116cm-lucky-star-vi-2-cai-mnls-364.html
Crawling: https://winmart.onl/combo-12-mut-rua-ly-tach-poly-tech-kz118-482.html
Crawling: https://winmart.onl/10-quyen-so-a3-bia-cung-6-tap-bang-dau-240-trang-5781.html
Crawling: https://winmart.onl/4-can-nuoc-giat-d-nee-thai-lan-3l-5358.html
Crawling: https://winmart.onl/2-chai-xi-dau-cang-cua-dac-biet-3079.html
Crawling: https://winmart.onl/150-cai-bao-dung-the-nhan-vien-108-2118.html
Crawling: https://winmart.onl/60-tap-truyen-tranh-tam-quoc-dien-nghia-3870.html
Crawling: https://winmart.onl/dung-phe-binh-toi-3766.html
Crawling: https://winmart.onl/2-chai-ruou-vang-hibiscus-roselle-914.html
Crawling: https://winmart.onl/ruou-ban-ngot-hibiscus-roselle-915.html
Crawling: https://winmart.onl/tam-suoi-dien-hk25-5493.html
Crawling: https://winmart.onl/quat-suoi-fairlady-nsb-90-cong-nghe-duc-3923.html
Crawling: https://winmart.onl/combo-8-chai-nuoc-giat-chong-phai-mau-kolortex-tang-2-chai-xa-vai-

In [42]:
df_product = pd.DataFrame(data)

# Lưu file CSV
df_product.to_csv("products/products.csv", index=False, encoding="utf-8")

print("Saved products.csv thành công!")

Saved products.csv thành công!
